**SmartReview** — RAG demo (Jupyter Notebook)

Цель: показать простую RAG-систему для аналитики отзывов:
- как загрузить датасет отзывов (`amazon_polarity`).
- как создать векторное хранилище Qdrant с индексом для фильтрации по настроению.
- как построить набор chain'ей/агентов для поиска жалоб, анализа причин и сравнения плюсов/минусов.

# loading data  into vector storage Qdrant

# import library

In [ ]:
# -------------------------
# Imports
# - Стандартные/утилитарные библиотеки
# - Работа с датасетами
# - LangChain / Qdrant / OpenAI — компоненты для RAG
# - Pydantic — строгость структуры маршрутизатора
# -------------------------
import os
from typing import List, Dict

# Библиотеки данных
from datasets import load_dataset

# LangChain и Qdrant
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# Провайдер embeddings и LLM (OpenAI)
from langchain_openai import OpenAIEmbeddings, ChatOpenAI

# Qdrant: векторное хранилище и клиент
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http import models # Важно для фильтрации и создания индексов!

# 1 cпособ задать переменные жоствко в коде как тут 
# --- КОНФИГУРАЦИЯ ---
# os.environ["OPENAI_API_KEY"] = ...

import os

os.environ['OPENAI_API_KEY'] = ...
os.environ["QDRANT_URL"] = ...
os.environ["QDRANT_API_KEY"] = ...
COLLECTION_NAME = "amazon_complaints"
def get_qdrant_client():
    return QdrantClient(url=os.environ['QDRANT_URL'], api_key=os.environ['QDRANT_API_KEY'], timeout=120)

In [ ]:
# 2 способ - это подгрузить переменые из окружения .env файла

# --- КОНФИГУРАЦИЯ ---
# from dotenv import load_dotenv
# load_dotenv()


# пример как будут зраниться переменные в .env файле
# OPENAI_API_KEY=sk-...
# QDRANT_URL=https://...
# QDRANT_API_KEY=eyJh...
# COLLECTION_NAME=amazon_complaints
# LANGSMITH_PROJECT=Amazon Product Analysis
# ...

In [ ]:
# --- 1. ЗАГРУЗКА ДАННЫХ (MIXED) ---

def load_balanced_data(limit_per_class=100):
    """
    Загружает сбалансированную выборку отзывов из датасета `amazon_polarity`.

    Параметры:
        limit_per_class: int
            Максимальное количество документов для каждого класса (positive/negative).

    Возвращает:
        List[Document] — список langchain Document с полями page_content и metadata.
            metadata содержит: original_title, sentiment ('positive'|'negative'), label (0|1).

    Примечания:
    - Используется streaming=True для экономии памяти.
    - Если датасет изменится, код может завершиться раньше, если не хватает примеров.
    """
    print(f"-> Загрузка данных (по {limit_per_class} шт. каждого типа)...")
    dataset = load_dataset("amazon_polarity", split="train", streaming=True)
    
    processed_docs = []
    neg_count = 0
    pos_count = 0
    
    for row in dataset:
        label = row['label'] # 0: neg, 1: pos
        
        # Логика балансировки
        if label == 0 and neg_count >= limit_per_class:
            continue
        if label == 1 and pos_count >= limit_per_class:
            continue
        
        sentiment = "negative" if label == 0 else "positive"
        # Формируем единый текстовый блок с названием и содержимым
        text_content = f"Title: {row['title']}\nReview: {row['content']}"
        
        # Создаем Document с нужными метаданными
        doc = Document(
            page_content=text_content,
            metadata={
                "original_title": row['title'],
                "sentiment": sentiment, # Ключевое поле для фильтрации
                "label": label
            }
        )
        
        # Считаем добавленные документы
        if label == 0:
            neg_count += 1
        else:
            pos_count += 1
            
        processed_docs.append(doc)
        
        if neg_count >= limit_per_class and pos_count >= limit_per_class:
            break
            
    print(f"Загружено {len(processed_docs)} документов (Neg: {neg_count}, Pos: {pos_count})")
    return processed_docs


In [ ]:
# Подготовка данных (балансированная)
docs = load_balanced_data(limit_per_class=500)

🔄 Загрузка данных (по 500 шт. каждого типа)...


'HTTPSConnectionPool(host='cas-bridge.xethub.hf.co', port=443): Read timed out.' thrown while requesting GET https://huggingface.co/datasets/amazon_polarity/resolve/9d9c45c18f8c3cf1b23a3c27917b60cbf28f3289/amazon_polarity/train-00000-of-00004.parquet
Retrying in 1s [Retry 1/5].
'(ReadTimeoutError("HTTPSConnectionPool(host='cas-bridge.xethub.hf.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 25afe6a5-9d19-4307-a906-d37dae0e56fe)')' thrown while requesting GET https://huggingface.co/datasets/amazon_polarity/resolve/9d9c45c18f8c3cf1b23a3c27917b60cbf28f3289/amazon_polarity/train-00000-of-00004.parquet
Retrying in 1s [Retry 1/5].
'HTTPSConnectionPool(host='cas-bridge.xethub.hf.co', port=443): Read timed out.' thrown while requesting GET https://huggingface.co/datasets/amazon_polarity/resolve/9d9c45c18f8c3cf1b23a3c27917b60cbf28f3289/amazon_polarity/train-00000-of-00004.parquet
Retrying in 2s [Retry 2/5].
'HTTPSConnectionPool(host='cas-bridge.xethub.hf.co', port=443): Read 

Загружено 1000 документов (Neg: 500, Pos: 500)


In [ ]:
len(docs)

1000

In [ ]:
# --- 2. ВЕКТОРНОЕ ХРАНИЛИЩЕ ---

def get_qdrant_client():
    """Инициализация клиента Qdrant"""
    return QdrantClient(url=os.environ['QDRANT_URL'], api_key=os.environ['QDRANT_API_KEY'], timeout=3600)

def setup_vector_store(docs: List[Document]):
    """
    Индексирует документы в Qdrant и создает payload index для фильтрации по sentiment.

    Поведение:
    - Создает коллекцию (force_recreate=True) — перезапишет предыдущую коллекцию с тем же именем.
    - Создаёт payload index для 'metadata.sentiment', чтобы можно было быстро фильтровать по 'positive'/'negative'.
    """
    client = get_qdrant_client()
    embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
    
    print(f"-> Индексация в коллекцию '{COLLECTION_NAME}'...")
    
    # 1. Загружаем векторы (это создает коллекцию, если её нет, или пересоздает при force_recreate)
    vector_store = QdrantVectorStore.from_documents(
        documents=docs,
        embedding=embeddings,
        url=os.environ['QDRANT_URL'],
        api_key=os.environ['QDRANT_API_KEY'],
        collection_name=COLLECTION_NAME,
        force_recreate=True
    )

    # 2. ВАЖНОЕ ИСПРАВЛЕНИЕ: Создаем Payload Index
    # Qdrant требует явного создания индекса для полей, по которым мы будем фильтровать
    print("-> Создание индекса для поля 'metadata.sentiment'...")
    
    client.create_payload_index(
        collection_name=COLLECTION_NAME,
        field_name="metadata.sentiment", # Путь к полю в JSON-структуре документа
        field_schema=models.PayloadSchemaType.KEYWORD # KEYWORD = точное совпадение (для строк типа 'positive'/'negative')
    )
    
    print("-> Индекс создан. Хранилище готово.")
    return vector_store


In [ ]:
# 2. Индексация
vector_store = setup_vector_store(docs)

🚀 Индексация в коллекцию 'amazon_complaints_2'...
⚙️ Создание индекса для поля 'metadata.sentiment'...
Индекс создан. Хранилище готово.


#AGENT

# Архитектура Агента: Semantic Routing
Вместо того чтобы задавать один общий промпт, мы создаем **специализированные цепочки (Skills)** для разных задач.

Мы реализуем паттерн **Router**:
1.  **Classifier (Router):** LLM анализирует вопрос пользователя и возвращает структурированный ответ (JSON), выбирая нужный инструмент (`complaints`, `root_cause`).
2.  **Specialized Chains:**

    - *`build_complaint_chain`* — retriever с фильтром negative, формирует краткий список проблем.
    - *`build_root_cause_chain`* — глубинный анализ, больше контекста (k=8).
    - *`build_comparative_chain`* — вытягивает позитивные и негативные отзывы и просит сделать таблицу плюсы/минусы.
    - *`build_general_chain`* — fallback, общий помощник.

## Создание Router Agent
Мы используем `with_structured_output` (Function Calling), чтобы заставить LLM выбрать один из предопределенных вариантов (`RouteQuery`). Это исключает галлюцинации при выборе инструмента.

In [ ]:

import os
from typing import List, Literal

# Библиотеки данных
from datasets import load_dataset

# LangChain и Qdrant
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http import models

# Pydantic — для структурированного вывода роутера
from pydantic import BaseModel, Field



import os

os.environ['LANGSMITH_ENDPOINT']=...
os.environ['LANGSMITH_API_KEY']=...
os.environ['OPENAI_API_KEY']=......
os.environ['TAVILY_API_KEY']=...
COLLECTION_NAME="amazon_complaints_2"
os.environ['LANGSMITH_PROJECT']="AMAZON product analysis"
os.environ["QDRANT_URL"]=...
os.environ['QDRANT_API_KEY']=...
os.environ["LANGCHAIN_TRACING"]="true" 


In [ ]:
# получение клиента Qdrant
def get_qdrant_client():
    return QdrantClient(url=os.environ['QDRANT_URL'], api_key=os.environ['QDRANT_API_KEY'], timeout=120)

# Получение LLM
def get_llm():
    return ChatOpenAI(model="gpt-4o-mini", temperature=0)

# --- 1. СТРОИТЕЛИ ЦЕПОЧЕК (CHAIN BUILDERS) ---

def build_complaint_chain(vector_store):
    """Story 1: Поиск жалоб (Negative Filter)"""

    # Жесткий фильтр на уровне базы данных
    filter_condition = models.Filter(
        must=[models.FieldCondition(key="metadata.sentiment", match=models.MatchValue(value="negative"))]
    )
    retriever = vector_store.as_retriever(search_kwargs={"k": 5, "filter": filter_condition})
    
    template = """Найди конкретные детали жалоб пользователей по теме вопроса.
    Контекст (только негативные отзывы): {context}
    Вопрос: {question}
    Ответ (кратко перечисли технические проблемы):"""
    
    return (
        {"context": retriever | (lambda docs: "\n".join([d.page_content for d in docs])), "question": RunnablePassthrough()}
        | ChatPromptTemplate.from_template(template)
        | get_llm()
        | StrOutputParser()
    ).with_config({"run_name": "ComplaintsChain"})

def build_root_cause_chain(vector_store):
    """Story 2: Глубинный анализ причин (Negative Filter + Deep Prompt)"""

    filter_condition = models.Filter(
        must=[models.FieldCondition(key="metadata.sentiment", match=models.MatchValue(value="negative"))]
    )
    retriever = vector_store.as_retriever(search_kwargs={"k": 8, "filter": filter_condition})
    
    template = """Ты — опытный Product Manager. Проанализируй отзывы и объясни, ПОЧЕМУ пользователи недовольны.
    Выяви паттерны (контроль качества, описание, доставка).
    
    Отзывы: {context}
    Тема: {question}
    
    Аналитический отчет:"""
    
    return (
        {"context": retriever | (lambda docs: "\n\n".join([d.page_content for d in docs])), "question": RunnablePassthrough()}
        | ChatPromptTemplate.from_template(template)
        | get_llm()
        | StrOutputParser()
    ).with_config({"run_name": "RootChain"})

def build_comparative_chain(vector_store):
    """Story 3: Сравнение (Parallel Retrieval)"""

    # Создаем два отдельных ретривера: для плюсов и для минусов
    neg_filter = models.Filter(must=[models.FieldCondition(key="metadata.sentiment", match=models.MatchValue(value="negative"))])
    neg_retriever = vector_store.as_retriever(search_kwargs={"k": 5, "filter": neg_filter})
    
    pos_filter = models.Filter(must=[models.FieldCondition(key="metadata.sentiment", match=models.MatchValue(value="positive"))])
    pos_retriever = vector_store.as_retriever(search_kwargs={"k": 5, "filter": pos_filter})
    
    template = """Сравни опыт пользователей по теме: {topic}.
    НЕГАТИВ: {neg_context}
    ПОЗИТИВ: {pos_context}
    Задача: Таблица "Плюсы vs Минусы" и вывод о покупке."""
    
    def format_list(docs):
        return "\n".join([f"- {d.page_content}" for d in docs])

    return (
        {
            "neg_context": neg_retriever | format_list,
            "pos_context": pos_retriever | format_list,
            "topic": RunnablePassthrough()
        }
        | ChatPromptTemplate.from_template(template)
        | get_llm()
        | StrOutputParser()
    ).with_config({"run_name": "ComparativeChain"})

def build_general_chain():
    """General Purpose Chain (No Filtering)"""

    prompt = ChatPromptTemplate.from_template(
        "Ты полезный ассистент SmartReview. Ответь пользователю вежливо.\nВопрос: {question}"
    )
    return prompt | get_llm() | StrOutputParser()

# --- 2. ROUTING LOGIC (МАРШРУТИЗАТОР) ---

# Описываем структуру ответа для классификатора, чтобы он не галлюцинировал
class RouteQuery(BaseModel):
    """Определяет, куда направить запрос пользователя."""
    destination: Literal["complaints", "root_cause", "comparison", "general"] = Field(
        ...,
        description="Выбери: 'complaints' (баги/жалобы), 'root_cause' (причины проблем), 'comparison' (сравнение/покупка) или 'general'."
    )

def create_router_agent(vector_store):
    llm = get_llm()
    
    # 1. Создаем "умного классификатора" с помощью structured output
    # Это заставляет LLM возвращать JSON с полем destination
    structured_llm_router = llm.with_structured_output(RouteQuery)
    
    system_prompt = """You are an expert router for a product analysis tool.
    Route the user's query to one of three destinations:
    
    1. 'complaints': Use this when the user asks "what is wrong", "find bugs", "list issues", "complaints about X".
    2. 'root_cause': Use this when the user asks "WHY is it bad", "explain the reason", "analyze the failure".
    3. 'comparison': Use this when the user asks "pros and cons", "should I buy", "compare good and bad", "is it worth it".
    
    Return only the destination."""
    
    router_prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("human", "{question}"),
    ])
    
    # Сама цепочка классификации
    router_chain = (router_prompt | structured_llm_router).with_config({"run_name": "RouterClassifier"}) # <-- В LangSmith будет видно это имя

    # 2. Создаем словарь всех наших инструментов
    chain_tools = {
        "complaints": build_complaint_chain(vector_store),
        "root_cause": build_root_cause_chain(vector_store),
        "comparison": build_comparative_chain(vector_store),
        "general": build_general_chain(),
    }

    # 3. Функция маршрутизации (принимает решение от LLM и запускает нужную цепочку)
    def route_logic(info):
        # info - это результат работы router_chain (объект RouteQuery)
        destination = info.destination
        print(f"-> Маршрутизатор выбрал: -> [{destination.upper()}]")
        return chain_tools[destination]

    # 4. Финальная "Магическая" цепочка
    # Сначала классифицируем -> передаем результат в route_logic -> запускаем выбранную цепочку
    full_chain =  {
        "destination": router_chain,
        "question": RunnablePassthrough()
    } | RunnableLambda(lambda x: route_logic(x["destination"]).invoke(x["question"]))

    return full_chain

# --- MAIN ---

def init_existing_vector_store():
    # client = get_qdrant_client()
    client = QdrantClient(
        url=os.environ['QDRANT_URL'],
        api_key=os.environ['QDRANT_API_KEY']
    )
    embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
    return QdrantVectorStore(
        client=client, 
        collection_name=COLLECTION_NAME, 
        embedding=embeddings
    )

if __name__ == "__main__":
    # Подключаемся к уже созданной базе
    vector_store = init_existing_vector_store()
    
    # Создаем агента-маршрутизатора
    agent = create_router_agent(vector_store)
    print("-> Агент готов! Введите вопрос (или 'exit' для выхода).")
    
    while True:
        user_query = input("\nYou: ")
        if user_query.lower() in ["exit", "quit"]:
            break
            
        try:
            response = agent.invoke(user_query)
            print(f"\nAgent:\n{response}")
        except Exception as e:
            print(f"Error: {e}")

🤖 Агент готов! Введите вопрос (или 'exit').
Примеры: 'What are the bugs with audio?', 'Should I buy this?', 'Why is delivery so bad?'
🚦 Router decided: Routing to -> [COMPARISON]

Agent:
Вот таблица "Плюсы vs Минусы" на основе представленных отзывов:

| Плюсы                                      | Минусы                                      |
|--------------------------------------------|---------------------------------------------|
| Хорошее качество сборки и упаковки         | Низкое разрешение и плохое качество видео   |
| Удобство использования                      | Неприятный запах резины                     |
| Доступная цена                             | Неэффективность в уменьшении объема         |
| Подходит для детей                         | Неправильный размер и форма                 |
| Удобные функции и простота в использовании  | Чувствуется дешевизна материала              |
| Безопасные материалы (для детей)          | Долгое время доставки                        |
|